<a href="https://colab.research.google.com/github/ASIF-Mahmud1/Exploration/blob/text-classifier/NLP_Lessons/BERT/EnronDatasetDiminsihingTerms_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Install torchvision
!pip install torchvision


In [3]:
import torch

In [4]:
from typing import Callable, List, Optional, Tuple

import pandas as pd
from sklearn.base import TransformerMixin, BaseEstimator
import torch


class BertTransformer(BaseEstimator, TransformerMixin):
    def __init__(
            self,
            bert_tokenizer,
            bert_model,
            max_length: int = 60,
            embedding_func: Optional[Callable[[torch.tensor], torch.tensor]] = None,
    ):
        self.tokenizer = bert_tokenizer
        self.model = bert_model
        self.model.eval()
        self.max_length = max_length
        self.embedding_func = embedding_func

        if self.embedding_func is None:
            self.embedding_func = lambda x: x[0][:, 0, :].squeeze()

    def _tokenize(self, text: str) -> Tuple[torch.tensor, torch.tensor]:
        # Tokenize the text with the provided tokenizer
        tokenized_text = self.tokenizer.encode_plus(text,
                                                    add_special_tokens=True,
                                                    max_length=self.max_length
                                                    )["input_ids"]

        # Create an attention mask telling BERT to use all words
        attention_mask = [1] * len(tokenized_text)

        # bert takes in a batch so we need to unsqueeze the rows
        return (
            torch.tensor(tokenized_text).unsqueeze(0),
            torch.tensor(attention_mask).unsqueeze(0),
        )

    def _tokenize_and_predict(self, text: str) -> torch.tensor:
        tokenized, attention_mask = self._tokenize(text)

        embeddings = self.model(tokenized, attention_mask)
        return self.embedding_func(embeddings)

    def transform(self, text: List[str]):
        if isinstance(text, pd.Series):
            text = text.tolist()

        with torch.no_grad():
            return torch.stack([self._tokenize_and_predict(string) for string in text])

    def fit(self, X, y=None):
        """No fitting necessary so we just return ourselves"""
        return self

In [5]:
!pip install transformers


     |████████████████████████████████| 2.2MB 5.7MB/s 
     |████████████████████████████████| 870kB 30.9MB/s 
     |████████████████████████████████| 3.3MB 41.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=ef1809ae445ff13892b032aca7a72b0d7a9cdb2e0266ca5d2a98d752e7cf41d4
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [6]:
from transformers import BertTokenizer, BertModel
import torch
bert_model = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [7]:
bert_transformer = BertTransformer(tokenizer, bert_model)
from sklearn.pipeline import Pipeline


In [8]:
url="https://raw.githubusercontent.com/ASIF-Mahmud1/Exploration/text-classifier/DiminishingTerms/dataSet.csv"
from io import StringIO
import string
import pandas as pd
import requests
s=requests.get(url).text

message_data=pd.read_csv(StringIO(s))
message_data['tag'] = message_data['tag'].str.strip()

message_data.head()
message_data['tag'].describe()

count             315
unique              2
top       diminishing
freq              234
Name: tag, dtype: object

In [9]:
message_data_copy = message_data['sentence'].copy() 

# message_data_copy

In [10]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs',C=5.2)


In [11]:
pipeline = Pipeline(steps=  [
        ("vectorizer", bert_transformer),
        ("classifier", classifier),
    ] )

pipeline.fit(message_data_copy,  message_data['tag'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_pa

Pipeline(memory=None,
         steps=[('vectorizer',
                 BertTransformer(bert_model=None, bert_tokenizer=None,
                                 embedding_func=<function BertTransformer.__init__.<locals>.<lambda> at 0x7faf5b206170>,
                                 max_length=60)),
                ('classifier',
                 LogisticRegression(C=5.2, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='multinomial', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [12]:
import pandas as pd
df = pd.read_csv('https://query.data.world/s/yqgskd2yg2lztwtgmpeajy75msskd6',error_bad_lines=False, index_col=False, dtype='unicode')
df = df.applymap(str) ## stringify all fields


In [19]:
sender= [
  "susan.bailey@enron.com",
	"sally.beck@enron.com",
  "lynn.blair@enron.com",
  "sandra.brawner@enron.com",
  "michelle.cash@enron.com",
  "monika.causholli@enron.com",
  "stacy.dickson@enron.com",
]

senderFinal=[]
for email in sender:
  email="frozenset({'" + email+"'})"
  senderFinal.append(email)


# femaleEmail= df.loc[df['From'].isin(senderFinal)]
# femaleEmail.head()

df= df.loc[df['From'].isin(senderFinal)]

print("Size of Dataset: ",df.shape[0])


Size of Dataset:  9167


In [20]:
collectionSize=10
startIndex=0
endIndex=startIndex + collectionSize

# Test Sentences Here
### **Note**: Click Play to Predict next 10 emails

In [23]:
####################################################################################################################
                       ## Remove this later
import pandas as pd
import altair as alt
import numpy as np
text= df['content']  # all data
# text= femaleEmail['content']    # all female data
text= text[startIndex:endIndex]
prob_Of_Each_Class = pipeline.predict_proba(text)
X = np.arange(startIndex,endIndex)
y_strong= prob_Of_Each_Class[:,1]*100
y_diminishing=prob_Of_Each_Class[:,0]*100    

dfTest = pd.DataFrame(columns=('index', 'strong', 'diminish','sentence'))
dfTest['index']=X 
dfTest['strong']=y_strong 
dfTest['diminish']=y_diminishing
dfTest['sentence']=list(text)
gp_chart = (
  alt.Chart(dfTest,width=50 ,padding={"left": 50, "top": 5, "right": 5, "bottom": 5} )
    .transform_fold(["diminish", "strong"], as_=["key", "value"])
    .mark_bar()  # size=50
    .encode(
        x="key:N",
        y="value:Q",
        color="key:N",
        column="index",
        tooltip="sentence"
    )   # .interactive() 
)

startIndex= startIndex+collectionSize
endIndex=startIndex+collectionSize
gp_chart.display()

    
for index, row in enumerate(text.index):
    print(X[index],  " "+text[row], " ",X[index])    # " ",row,
    
                            ## Remove this later
####################################################################################################################

alt.Chart(...)

20  Patrick, Please furnish a hard copy of each of the swap confirmations for: Deal Nos. V48090.1 & V48242.1 These deals were referenced by the Counterparty. However, there may be additional confirmations for other financial transactions, therefore, please those confirmations as well (if any). Thank for all the help. Cordially, Susan S. Bailey Enron North America Corp. 1400 Smith Street, Suite 3803A Houston, Texas 77002 Phone: (713) 853-4737 Fax: (713) 646-3490 Email: Susan.Bailey@enron.com   20
21  Ellen, Please check to see if there are any "live" physical deals with the captioned counterparty. If so, please provide a hard copy of the confirmation(s) evidencing such deals. I wuld appreciate receipt of such confirmations by Wednesday, February 6th. Many thanks for all your help. Cordially, Susan S. Bailey Enron North America Corp. 1400 Smith Street, Suite 3803A Houston, Texas 77002 Phone: (713) 853-4737 Fax: (713) 646-3490 Email: Susan.Bailey@enron.com   21
22  Ellen, Yes, that is the